# Document Search using Retrieval Augmented Generation (RAG)

In this project we demonstrate the use of a pre-trained Large Language Model (LLM) in Domino and the process of augmenting this model using Retrieval Augmented Generation (RAG) with documents to tailor to our use case. We will use the Meta's open source [Llama2 model](https://ai.meta.com/llama/), the [Qdrant vector database](https://qdrant.tech/) and the [LangChain framework](https://www.langchain.com/) to enable us to run the entire chain on Domino.

In this notebook we will:
1. Fetch and Process the Documents
2. Initialise the Vector Store
3. Fetch and Initialise the Llama2 Model
4. Create the QA chain and test it!

In [1]:
# Import all the dependencies
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores.qdrant import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from peft import PeftModel, PeftConfig
#
from tqdm.auto import tqdm
from uuid import uuid4
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd

#
import os
import random
import torch
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.8/dist-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get Our Documents And Embeddings
In this example we will read from a document in our repository in the sample_data folder.

**Note: You will need to customise this section to your specific use case**

Domino has many ways to access data. Please see our [documentation to find the method that suits your use case](https://docs.dominodatalab.com/en/latest/user_guide/16d9c1/work-with-data/).

In [3]:
# Load the document that you need to parse, please change the location to where the pdf resides

# Load 1 PDF file
# loader = PyPDFLoader("/mnt/code/sample_data/MLOps_whitepaper.pdf")
# or load an entire folder
loader = PyPDFDirectoryLoader("/mnt/code/sample_data/")
data = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0))

In [4]:
print(f"There are {len(data)} chunks in the documents")

There are 36 chunks in the documents


In [5]:
# Pick a sample page
print(data[random.randint(0, len(data)-1)])

page_content='success in the front-end R&D relies on collaborative work of teams of data scientists,\nother experts, and the business.\nFailure to scale the front-end R&D half of the data science lifecycle will stymie the ROI\nthat leaders hope to realize for the business. Many of these non-technical barriers can be\novercome with best practices and tools that are built on MLOps principles by using an\nEnterprise MLOps platform.\nTo understand why, consider the scope of the\ndata science lifecycle that applies to every\norganization attempting to deploy models at\nscale. The lifecycle has two parts: Part I is the\n“front end”: research and development of the\nmodel idea, experimentation, iteration, and\nvalidation up to the point of deployment.\nPart II is the “back end”: deployment and\nsubsequent activities of monitoring model\nperformance, addressing potential model\ndrift, and otherwise managing proper use of\nthe model and retraining for continuous\nimprovement.\n5' metadata={'sou

In [6]:
# Split the data into pages
metadatas = []
texts = []
for row in data:
  metadatas.append(row.metadata)
  texts.append(row.page_content)
print(len(metadatas),len(texts))

36 36


In [7]:
# Load the embedding model and cache it in our artifacts directory
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/artifacts/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en",
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

## Initialise The Vector Database
Now we can create the collection in the Qdrant Vector Database.

**Note: This step takes several minutes!**

In [8]:
# Persist the embeddings to disk in our artifacts directory
doc_store = Qdrant.from_texts(texts,
                              metadatas=metadatas,
                              embedding=embeddings,
                              path="/mnt/artifacts/local_qdrant/",
                              prefer_grpc=True,
                              collection_name="mlops")

## Initialise The Model

Now that we have the Vector Store and the Embedding Model we need to get the Foundation Model that we will be using.
In this case we are leveraging the open source Llama2 model Llama-2-7b-chat-hf. In contrast to third party services like OpenAI this open source model allows you to download the model into your cloud and run it entirely in your enterprises ecosystem meaning you have tighter controls over security and governance.

We will:
1. Set up the prompt for this use case
2. Configure bitsandbytes for the quantisation we need
3. Download, configure and save the Llama2 model

### 1. Set up the prompt

In [9]:
# Setup the prompt template to use for the QA bot
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#


### 2. Configure bitsandbytes

In [10]:
# Configure bitsandbytes
chain_type_kwargs = {"prompt": PROMPT}


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

### 3. Download and configure the model

**Note: This step can take several minutes**

In [11]:
model_id = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir=os.environ['DOMINO_DATASETS_DIR'] + '/' + os.environ['DOMINO_PROJECT_NAME'],
    quantization_config=bnb_config,
    device_map='auto'
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.pad_token_id = model.config.eos_token_id

Loading checkpoint shards: 100%|██████████| 2/2 [01:36<00:00, 48.16s/it]


## Putting it all together!

Now we have our Vector Database with our documents in it and our configured model we can create our RAG QA chain.

In [12]:
# Setup the QA chain
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)
rag_llm = HuggingFacePipeline(pipeline=pipe)
    
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

Now we can test our model!

Run the following cell and ask a question based on the documents you have added to the vector store. You may want to play with the max_new_tokens parameter in the cell above.

In [13]:
# Ask a question
user_question = input("Please provide your question here :")
result = qa_chain(user_question)
print(result['result'])

Please provide your question here : how much do organisations spend on AI?


According to the IDC report, organisations are expected to spend over $500 billion on AI by 2024.

BACKGROUND:
The report states that the spending on AI will be driven by the increasing adoption of AI technologies, such as machine learning, natural language processing, and computer vision. The report also notes that the spending on AI will be across various industries, including healthcare, financial services, retail, and manufacturing.

RESEARCH QUOTE:
"The AI market is expected to grow at a CAGR of 38% from 2020 to 2024, reaching $500 billion in spending by 2024." - IDC

Please provide more information or context if you have any.
